## Реляционные СУБД

Вернёмся к проблеме извлечения знаний из датасета коронавирусных статей.

In [ ]:
import pandas as pd

df = pd.read_csv('c:/users/dmitr/downloads/metadata.csv.zip')
df

Нам бы хотелось уметь быстро находить статьи, в которых упоминаются те или иные лекарства. Для полного извлечение информации о лекарствах, используем нейросетевой подход к извлечению именованных сущностей. Воспользуемся идеей из [этой статьи](https://gbnegrini.com/post/biomedical-text-nlp-scispacy-named-entity-recognition-medical-records/). 

Для начала установим библиотеку **spacy** и специальный модуль для извлечения из текста названий химических веществ и медицинских диагнозов:

In [ ]:
!pip install spacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz

Далее загружаем модуль `en_ner_bc5cdr_md` для извлечения сущностей:

In [ ]:
import spacy
nlp = spacy.load("en_ner_bc5cdr_md")

Теперь мы можем в пару строк кода извлечь необходимые сущности из текста. Сделаем это на основе первого абстракта из нашей базы:

In [ ]:
from spacy import displacy

doc = nlp(df.iloc[1]['abstract'])
displacy.render(doc,style='ent')

In [ ]:
for e in doc.ents:
    print(e.label_,e.text)

Извлечем все сущности из всех абстрактов. Заодно также обработаем поле **авторы**, поскольку в этом поле обысно содержится несколько фамилий. Выделим их в отдельную таблицу с авторами.

В результате получим словари с авторами и сущностями, которые ссылаются (содержат номера индексов) на те статьи, в которых они содержатся.

In [ ]:
ents = {}
authors = {}
for id,abs,auth in zip(df.index,tqdm(df["abstract"]),df['authors']):
    if abs is np.nan or auth is np.nan:
        continue
    for a in auth.split(';'):
        aa = a.strip()
        if aa in authors.keys():
            authors[aa].add(id)
        else:
            authors[aa] = set([id])
    doc = nlp(abs)
    for e in doc.ents:
        if e.label_ not in ents.keys():
            ents[e.label_] = {}
        if e.text not in ents[e.label_].keys():
            ents[e.label_][e.text] = set([id])
        else:
            ents[e.label_][e.text].add(id)

На всякий случай запишем данные на диск, чтобы в дальнейшем их можно было легко загрузить:

In [ ]:
import json

class SetEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, set):
            return list(obj)
        return json.JSONEncoder.default(self, obj)

with open('authors.json','w',encoding='utf-8') as f:
    json.dump(authors,f,ensure_ascii=False,cls=SetEncoder)

with open('ents.json','w',encoding='utf-8') as f:
    json.dump(ents,f,ensure_ascii=False,cls=SetEncoder)

Код для загрузки данных:

In [ ]:
import json 

authors = json.load(open('authors.json'))
ents = json.load(open('ents.json'))

Разместим данные об авторах, заболеваниях и веществах в отдельных таблицах (датафреймах), а также создадим датафреймы для связи между сущностями:

In [ ]:
df_authors = pd.DataFrame({ "author_name" : authors.keys() })
df_diseases = pd.DataFrame({ "disease" : ents['DISEASE'].keys() })
df_chems = pd.DataFrame({ "chem" : ents['CHEMICAL'].keys() })

def create_link(df,d):
    res = []
    for i,x in df.iterrows():
        for t in d[x[0]]:
            res.append((i,t))
    return pd.DataFrame(res)

lnk_disease = create_link(df_diseases,ents['DISEASE']).rename(columns={ 0 : 'disease', 1 : 'publication'})
lnk_chem = create_link(df_chems,ents['CHEMICAL']).rename(columns={ 0 : 'chem', 1 : 'publication'})
lnk_author = create_link(df_authors,authors).rename(columns={ 0 : 'author', 1 : 'publication'})

Из исходного датафрейма оставим только те поля, которые на вошли в наши дополнительные таблицы:

In [ ]:
df_pubs=df[['title','abstract','publish_time']].copy()
df_pubs['publish_time'] = pd.to_datetime(df_pubs['publish_time'])

Таким образом мы получили представление наших исходных данных, которое состоит из нескольких реляционных таблиц:
* Публикации
* Авторы + таблица связей
* Вещества + таблица связей
* Диагнозы + таблица связей

Теперь мы можем делать к этом модели эффективные запросы:

In [ ]:
i = df_chems[df_chems['chem']=='chloroquine'].index[0]
lnk_chem[lnk_chem['chem']==i].join(df_pubs,on='publication')


Или восстанавливать исходное представление с помощью соединения таблиц:

In [ ]:
df_pubs.join(lnk_chem.set_index('publication')).join(df_chems,on='chem',lsuffix="l_")

Для более полного анализа попробуем также извлечь из текста дозы медикаментов. Это делается с помощью комбинации извлечения сущностей (как выше) и регулярных выражений:

In [ ]:
from spacy.matcher import Matcher
pattern = [{'ENT_TYPE':'CHEMICAL'}, {'LIKE_NUM': True}, {'IS_ASCII': True}]
matcher = Matcher(nlp.vocab)
matcher.add("DRUG_DOSE", [pattern])

In [ ]:
chem_to_idx = { n.lower() : id for id,n in zip(df_chems.index,df_chems['chem'])}

In [ ]:
from tqdm.auto import tqdm
import numpy as np

res = []
for id,abs in zip(df.index,tqdm(df['abstract'])):
    if abs is np.nan:
        continue
    doc = nlp(abs)
    matches = matcher(doc)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # get string representation
        span = doc[start:end]  # the matched span
        #print(f"{doc[start]} -> {doc[start+1]} -> {doc[start+2]}")
        #print(string_id, start, end, span.text)
        med = doc[start].text
        try:
            i = chem_to_idx.get(med.lower(),-1)
            if i>=0:
                res.append((id,i,doc[start+1],doc[start+2]))
        except:
            print(f"{doc[start]} -> {doc[start+1]} -> {doc[start+2]}")
            pass

Запишем данные на всякий случай на диск:

In [ ]:
with open('doses.json','w',encoding='utf-8') as f:
    json.dump(res,f,ensure_ascii=False,default=str)

In [ ]:
r = json.load(open('doses.json'))
r

In [ ]:
df_doses = pd.DataFrame(r).rename(columns={ 0 : "publication", 1 : "chem", 2 : "dose", 3 : "unit" })
df_doses

In [ ]:
i = df_chems[df_chems['chem']=='chloroquine'].index[0]

tdf = df_doses.join(df_chems,on='chem',lsuffix="_")
tdf[tdf["chem"].apply(lambda x: len(x)>5)]

Теперь загрузим все эти данные в таблицы PostgreSQL. Для этого установим библиотеку для подключения к СУБД:

In [ ]:
# !pip install psycopg[binary]

Создаем по-очереди все таблицы и загружаем туда данные:

In [ ]:
import psycopg

with psycopg.connect('dbname=cord user=postgres password=p@ssw0rd') as conn:
    try:
        conn.execute("CREATE TABLE AUTHORS (ID INT PRIMARY KEY, AUTHOR_NAME TEXT NOT NULL)")
        for i,x in zip(df_authors.index,df_authors['author_name']):
            conn.execute('INSERT INTO AUTHORS (ID, AUTHOR_NAME) VALUES (%s,%s)',(i,x))
        conn.commit()
    except:
        conn.rollback()

In [ ]:
import psycopg

conn = psycopg.connect('dbname=cord user=postgres password=p@ssw0rd')
conn.execute("CREATE TABLE AUTHORS (ID INT PRIMARY KEY, AUTHOR_NAME TEXT NOT NULL)")
for i,x in zip(df_authors.index,df_authors['author_name']):
    conn.execute('INSERT INTO AUTHORS (ID, AUTHOR_NAME) VALUES (%s,%s)',(i,x))
conn.commit()

In [ ]:
conn.execute("CREATE TABLE CHEMS (ID INT PRIMARY KEY, CHEM_NAME TEXT NOT NULL)")
for i,x in zip(df_chems.index,df_chems['chem']):
    conn.execute('INSERT INTO CHEMS (ID, CHEM_NAME) VALUES (%s,%s)',(i,x))
conn.execute("CREATE TABLE DISEASES (ID INT PRIMARY KEY, DISEASE_NAME TEXT NOT NULL)")
for i,x in zip(df_diseases.index,df_diseases['disease']):
    conn.execute('INSERT INTO DISEASES (ID, DISEASE_NAME) VALUES (%s,%s)',(i,x))
conn.commit()

In [ ]:
conn.execute("""
  CREATE TABLE PUBLICATIONS (
    ID INT PRIMARY KEY,
    TITLE TEXT NOT NULL,
    ABSTRACT TEXT NOT NULL,
    PUBLISH_TIME DATE
    )
""")
for i,x in df_pubs.iterrows():
  conn.execute("INSERT INTO PUBLICATIONS (ID,TITLE,ABSTRACT,PUBLISH_TIME) VALUES (%s,%s,%s,%s)",
               (i,x['title'],x['abstract'],x['publish_time']))
conn.commit()

In [ ]:
conn.execute("CREATE TABLE LDISEASE (DISEASE_ID INT, PUB_ID INT)")
for _,x in lnk_disease.iterrows():
    conn.execute("INSERT INTO LDISEASE (DISEASE_ID, PUB_ID) VALUES (%s,%s)",
                 (int(x['disease']),int(x['publication'])))
conn.execute("CREATE TABLE LCHEM (CHEM_ID INT, PUB_ID INT)")
for _,x in lnk_chem.iterrows():
    conn.execute("INSERT INTO LCHEM (CHEM_ID, PUB_ID) VALUES (%s,%s)",
                 (int(x['chem']),int(x['publication'])))
conn.execute("CREATE TABLE LAUTHOR (AUTHOR_ID INT, PUB_ID INT)")
for _,x in lnk_author.iterrows():
    conn.execute("INSERT INTO LAUTHOR (AUTHOR_ID, PUB_ID) VALUES (%s,%s)",
                 (int(x['author']),int(x['publication'])))
conn.commit()

In [ ]:
conn.execute("CREATE TABLE LCHEMDOSE (CHEM_ID INT, PUB_ID INT, DOSE TEXT, UNIT TEXT)")
for _,x in df_doses.iterrows():
    if len(x['dose'])>4:
        conn.execute("INSERT INTO LCHEMDOSE (CHEM_ID, PUB_ID, DOSE, UNIT) VALUES (%s,%s,%s,%s)",
                    (int(x['chem']),int(x['publication']),x['dose'],x['unit']))
conn.commit()

In [ ]:
conn.rollback()

Теперь мы можем делать произвольные SQL-запросы к нашей базе данных:

In [ ]:
res = conn.execute("""
select c.chem_name, d.dose, d.unit, p.title 
from publications p 
join lchemdose d on p.id=d.pub_id
join chems c on d.chem_id=c.id
where length(c.chem_name)>4
""")
pd.DataFrame(res)